### 오일러 피 함수
- $n$ 보다 작고 $n$ 과 서로소인 자연수의 개수를 구하는 함수
- $n$ 이 소수일 때, $\phi(n) = n - 1$ 이다. 
  - 소수일 경우 약수가 1과 자기 자신밖에 없으므로, 자신을 제외한 모든 수와 서로소이다. 따라서 $n - 1$
- 어떤 수가 소수인 수 $p$ 의 거듭제곱 꼴일 때 $\phi(p^k) = p^k - p^{k-1}$ 이다.
  - 소수 $p$ 의 거듭제곱과 서로소가 되려면, $p$ 의 배수가 아니어야 한다. \
  따라서 $p^k$ 에서 $p$ 의 배수인 수는 $p, 2p, 3p, ..., p^{k-1}, p^{k-1}p$ 가 있고, 이러한 개수가 $p^{k-1}$ 개이다.
  - 또한 $\phi(p^a) = p^a - p^{a-1} = p^{a-1}(p - 1) = p^a(1 - \frac{1}{p})$ 이다.
- 두 자연수 $m$ 과 $n$ 이 서로소라면, $\phi(mn) = \phi(m) \times \phi(n)$ 이다.
- https://mathphysics.tistory.com/512
- https://rkm0959.tistory.com/181

### 구현
- 시간복잡도: $O(N \log \log N)$
  - $N \log N$ 보다 느리게 증가하며, 에라토스테네스의 체의 시간복잡도와 같다.
- 공간복잡도: $O(N)$
- 즉 사실상 $10^7$ 정도 까지만 쓸 수 있다. 그보다 큰 숫자는 수학적으로 다른 분석이 필요할 것이다.
  - 11689(GCD(n, k) = 1)

In [ ]:
def phi(n):
  L = [i if i & 1 else i // 2 for i in range(n + 1)]
  for i in range(3, n + 1, 2):
    if L[i] != i: continue
    for j in range(i, n + 1, i):
      L[j] = (L[j] // i) * (i - 1)

  return L

### 뤼카의 정리
- $0 < n, r$ 인 정수와 소수 $p$ 에 대해 뤼카의 정리는 다음과 같은 합동식으로 표현할 수 있다.
- $\displaystyle\binom{n}{r} \equiv \prod^k_{i=0} \binom{n_i}{r_i}\pmod p$
- $n, r$ 을 $p$ 진법으로 나타낸다면
  - $n = n_kp^k + n_{k-1}p^{k-1} + \cdots + n_1p + n_0$
  - $r = r_kp^k + r_{k-1}p^{k-1} + \cdots + r_1p + r_0$
  - $0 \le i \le k$ 에 대해 $n_i$ 에서 $r_i$ 를 고르는 조합의 수, 즉 $\displaystyle\binom{n_i}{r_i}$ 를 모두 곱하면 $\displaystyle\binom{n}{r}$ 을 구할 수 있다.

### 중국인의 나머지 정리(CRT)
- 서로소인 n개의 수에 대해 일정한 나머지를 만족하는 수를 찾는 문제
- 모든 $i \ne j$ 에 대해 $\gcd(m_i, m_j) = 1$ 일 때, 연립부등식 \
$\begin{cases} x \equiv {n_1} \pmod {m_1} \\ x \equiv {n_2} \pmod {m_2} \\ \vdots \\ x \equiv {n_k} \pmod {m_k} \end{cases}$ 의 해는
- $M = \prod^k_im_i, M_i = \frac{M}{m_i} $ 일 때\
$\displaystyle x \equiv \sum_{i=1}^k M_i ( M_{i}^{-1} \pmod {m_i} ) n_i \pmod M $ 이다
- 문제에서 나눠야할 수들이 서로소가 아닐경우 모듈러들끼리 소인수분해 해서 각 소인수에 대한 항등식을 따로따로 만들면 된다.

In [ ]:
import math
import operator as op
from functools import reduce
lcm = lambda a, b: a * b // math.gcd(a, b)
def crt(a, p):
  prod = reduce(op.mul, p, 1)
  x = [prod // pi for pi in p]
  return sum(a[i] * pow(x[i], p[i] - 2, p[i]) * x[i] for i in range(len(a))) % prod

def xGCD(a, b):
  s, _s = 0, 1
  r, _r = b, a
  while r:
    q = _r // r
    _r, r = r, _r - q * r
    _s, s = s, _s - q * s
  return _r, _s, (_r - _s * a) // b if b else 0

def composite_crt(b, m):
  """returns x s.t. x = b[i] (mod m[i]) for all i"""
  x, m_prod = 0, 1
  for bi, mi in zip(b, m):
    g, s, _ = xGCD(m_prod, mi)
    if ((bi - x) % mi) % g:
      return None
    x += m_prod * (s * ((bi - x) % mi) // g)
    m_prod = (m_prod * mi) // math.gcd(m_prod, mi)
  return x % m_prod